In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

In [35]:
class BeamSearch(nn.Module):
    '''
    Implement BeamSearch for testing
    '''
    
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, src_len, tgt, tgt_len, width, upper_limit):
        max_len = tgt.size(0)
        vocab_size = self.decoder.dim_output
#         outputs = Variable(torch.zeros(max_len, width, vocab_size)).cuda()
        encoder_output, hidden = self.encoder(src, src_len)
        hidden = hidden[:self.decoder.num_layers]
        
        #output stores (probability, vector)
        output = Variable(torch.zeros(width, tgt.size(0))).cuda()
        upperBound = Variable(torch.zeros(upper_limit, tgt.size(0))).cude()
        
        prob = torch.tensor(0)
        #start with one word with probability of 1
        output[0][0] = 1
        output[0][1] = tgt.data[0, :]
        
        for t in range(1, max_len):
            idx = 0
            for i in range(width):
                #probability of previous vector
                prob = output[i][1].data
                
                output[i][1], hidden, attn_weights = self.decoder(
                        output[i][1], hidden, encoder_output)
                
                #update at most upper_limit number of possible vector
                for item in output[i][1].data:
                    upperBound[idx][1] = item
                    upperBound[idx][0] = item.data * (prob + 1e-9)
                    idx += 1
                    if idx >= upper_limit:
                        break
                
                #keep the width size
                val, idx = torch.topk(upperBound, k = width, dim = 0)
                sorted_idx, new_idx = torch.sort(idx, dim=-1)
                output = torch.gather(val, dim=-1, index=new_idx)    

        return output.data

In [31]:
# x = torch.rand(2, 3, 5)
# val, idx = torch.topk(x, k=2, dim=-1) # k greatest elements
# sorted_idx, new_idx = torch.sort(idx, dim=-1)
# val = torch.gather(val, dim=-1, index=new_idx)